<a href="https://colab.research.google.com/github/marcusvpimentel/Mestrado/blob/main/Dataset_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

## Fazer dowload dos arquivos

In [ ]:
import requests
import tarfile

url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = 'aclImdb_v1.tar.gz'

# Baixa o arquivo
response = requests.get(url, stream=True)
with open(filename, 'wb') as f:
    f.write(response.raw.read())

# Descompacta o arquivo
with tarfile.open(filename, 'r:gz') as tar:
    tar.extractall()


## Carregar os dados

In [ ]:
import os

def load_imdb_data(data_dir, subset='train'):
    data = {'texts': [], 'labels': []}
    for label in ['pos', 'neg']:
        labeled_dir = os.path.join(data_dir, subset, label)
        for file_name in os.listdir(labeled_dir):
            if file_name.endswith('.txt'):
                with open(os.path.join(labeled_dir, file_name), encoding='utf-8') as f:
                    data['texts'].append(f.read())
                    data['labels'].append(1 if label == 'pos' else 0)
    return data

# Carrega os dados de treino
data_dir = 'aclImdb'
train_data = load_imdb_data(data_dir, 'train')


## Pipeline

In [ ]:
from transformers import pipeline

# Função para truncar textos longos
def truncate_text(text, max_length=512):
    return text[:max_length]

# Cria a pipeline de análise de sentimentos
#sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english', truncation=True)
sentiment_analyzer = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True)

# Trunca os textos longos
truncated_texts = [truncate_text(text) for text in train_data['texts']]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# Analisa os sentimentos dos textos
# Processamento em lotes
batch_size = 32
results = []
print(len(truncated_texts))
for i in range(0, len(truncated_texts), batch_size):
    batch_texts = truncated_texts[i:i+batch_size]
    print(i)
    batch_results = sentiment_analyzer(batch_texts)
    results.extend(batch_results)

results[0]

25000
0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560
6

{'label': 'POSITIVE', 'score': 0.9986442923545837}

In [ ]:
results[0]['label']

'POSITIVE'

## Analisar e armazenar os resultados

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Extrai as previsões
predictions = [1 if results[0]['label'] == 'POSITIVE' else 0 for result in results]

# Calcula as métricas
accuracy = accuracy_score(train_data['labels'], predictions)
precision = precision_score(train_data['labels'], predictions)
recall = recall_score(train_data['labels'], predictions)
f1 = f1_score(train_data['labels'], predictions)

print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')


Acurácia: 0.5
Precisão: 0.5
Recall: 1.0
F1-score: 0.6666666666666666
